# Urban rural sensors

This notebook classifies sensors by whether they are in urban or rural areas.

In [ ]:
using CSV
using Glob
using DataFrames
using Missings
using ProgressBars
using ArchGDAL

In [ ]:
sensor_meta = CSV.read("../data/good_sensors.csv", DataFrame)

In [ ]:
names(sensor_meta)

In [ ]:
uzas = DataFrame(ArchGDAL.getlayer(ArchGDAL.read("../data/tl_2020_us_uac10.shp"), 0))
nothing

In [ ]:
sensor_meta[ismissing.(sensor_meta.Longitude),:]

In [ ]:
sensor_meta.geom = passmissing(ArchGDAL.createpoint).(sensor_meta.Longitude, sensor_meta.Latitude)
nothing

In [ ]:
names(uzas)

In [ ]:
# filter to just California, and just urbanized areas rather than urban clusters
ca_uza_geom = uzas[occursin.(["CA"], uzas.NAME10) .& (uzas.UATYP10 .== "U"), ""]
nothing

In [ ]:
@assert !any(ismissing.(sensor_meta.geom))

In [ ]:
sensor_meta.urban = map(sensor_meta.geom) do g
    for poly in ca_uza_geom
        if ArchGDAL.contains(poly, g)
            return true
        end
    end    
    return false  # not in any UZA
end
nothing

In [ ]:
sum(sensor_meta.urban)

In [ ]:
sum(.!sensor_meta.urban)

In [ ]:
@assert !any(ismissing.(sensor_meta.urban))

In [ ]:
CSV.write("../data/sensor_meta_geo.csv", sensor_meta)